# Converting text and code in questions and answers
This notebook loads a jsonl file that contains a list of dictionaries of the following format:
```
{
    "imports":"from skimage.io import imread\nimport stackview",
    "text":"You can load and show an image like this",
    "code":"image = imread('blobs.tif')\nstackview.insight(image)"
}
```

It will turn this information into a list of dictionaries in this format:
```
{
    "question:":"How can I open and visualize blobs.tif ?",
    "answer":"You can do this like this:\n\n```python\n```from skimage.io import imread\nimport stackview\n\nimage = imread('blobs.tif')\nstackview.insight(image)"
}
```

This is done using a languge model. Executing this notebook for about 70 entries costs about $0.10.

In [1]:
import json
import openai
import time
from bia_bob import bob
import os
from bia_bob._utilities import filter_out_blacklist, save_jsonl_file, load_jsonl_file
import json
import ipywidgets as widgets
from IPython.display import display
from bia_bob._utilities import load_jsonl_file, save_jsonl_file

In [2]:
training_data = load_jsonl_file("cle_imports_text_code_selected.jsonl")
training_data[0]

{'imports': 'import pyclesperanto_prototype as cle\n\nfrom skimage.io import imread, imsave, imshow\nimport matplotlib\nimport numpy as np\n\n# initialize GPU\ncle.select_device("GTX")',
 'text': '\n\n## Crop out a part of the image',
 'code': 'width = 75\nheight = 75\ntile = cle.create([height, width])\n\nx = 10\ny = 10\ncle.crop(image, tile, x, y)\n\ntile'}

In [3]:
def prompt(message:str, model="gpt-3.5-turbo"):
    """A prompt helper function that sends a message to openAI
    and returns only the text response.
    """
    import openai
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "user", "content": message}]
    )
    return response['choices'][0]['message']['content']

In [4]:
def make_question_answer(imports, text, code):
    question = prompt("Reformulated the following so that it becomes a question asking for Python code.\n\n" + text)
    refined_code = prompt(f"""
Add necessary import statements to Python code so that the code works. 
Do not add import statements that are not necessary.
These are the available import statements:
```
{imports}
```

And this is the code:
```
{code}
```

Keep the original code, add import statements as explained above and respond with code only.
""")
    refined_code = refined_code.replace("```python", "").replace("```", "")
    
    
    explanation = prompt(f"""Explain the following code shortly, but do not explain import statements:
```python
{refined_code}
```
""")

    answer = f"""
{explanation}

```python
{refined_code}
```
""" 

    return question, answer

#q, a = make_question_answer(training_data[0]['imports'], training_data[0]['text'], training_data[0]['code'])
#
#print(f"""
#Q: {q}
#
#A: {a}
#""")

In [5]:
try:
    questions_answers = load_jsonl_file("cle_questions_answers.jsonl")
except:
    questions_answers = []
for i, entry in enumerate(training_data):
    if i < len(questions_answers):
        continue
    print(i)
    q,a = make_question_answer(entry['imports'], entry['text'], entry['code'])
    questions_answers.append({
        "question":q,
        "answer":a,
    })

    save_jsonl_file(questions_answers, "cle_questions_answers.jsonl")

    # do not overheat the OpenAI API
    # time.sleep(10)

78
79
80
81
82
83
84
85
86
87
88
89
90
91
